# Bab 8 Backpropagation (1)

## Praktikum

### a) Fungsi *Binary Encoding* dan *Decoding*

Tulis kode ke dalam *cell* di bawah ini:

In [ ]:
def bin_enc(lbl):
    mi = min(lbl)
    length = len(bin(max(lbl) - mi + 1)[2:])
    enc = []

    for i in lbl:
            b = bin(i - mi)[2:].zfill(length)
            enc.append([int(n) for n in b])
    return enc

def bin_dec(enc, mi=0):
    lbl = []

    for e in enc:
        rounded = [int(round(x)) for x in e]
        string = ''.join(str(x) for x in rounded)
        num = int(string, 2) + mi
        lbl.append(num)
    return lbl

### b) Percobaan *Binary Encoding* dan *Decoding*

Tulis kode ke dalam *cell* di bawah ini:

In [ ]:
del print
labels = 1, 2, 3, 4
enc = bin_enc(labels)
dec = bin_dec(enc, min(labels))

print(enc)
print(dec)

[[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1]]
[1, 2, 3, 4]


### c) Fungsi *One-hot Encoding* dan *Decoding*

Tulis kode ke dalam *cell* di bawah ini:


In [ ]:
import numpy as np

def onehot_enc(lbl, min_val=0):
    mi = min(lbl)
    enc = np.full((len(lbl), max(lbl) - mi + 1), min_val, np.int8)

    for i, x in enumerate(lbl):
        enc[i, x - mi] = 1
    return enc

def onehot_dec(enc, mi=0):
    return [np.argmax(e) + mi for e in enc]

### d) Percobaan *Binary Encoding* dan *Decoding*

Tulis kode ke dalam *cell* di bawah ini:

In [ ]:
labels = 1, 2, 3, 4
enc = onehot_enc(labels)
dec = onehot_dec(enc, min(labels))

print(enc)
print(dec)

[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]
[1, 2, 3, 4]


### e) Fungsi Aktivasi Sigmoid dan Derivatifnya

Tulis kode ke dalam *cell* di bawah ini:

In [ ]:
def sig(X):
    return [1 / (1 + np.exp(-x)) for x in X]

def sigd(X):
    output = []

    for i, x in enumerate(X):
        s = sig([x])[0]
        output.append(s * (1 - s))
    return output

## Soal-soal
1. Download dataset Iris dalam format CSV di
https://datahub.io/machine-learning/iris/r/iris.csv .
2. Baca kolom terakhir pada file tersebut yang berisi kelas data. Buatlah
variabel bernama kelas dengan tipe list of string. Variabel kelas berisi
semua kelas yang terdapat pada file CSV tersebut.
3. Buatlah fungsi bernama bin_enc_str yang berfungsi untuk melakukan
binary encoding pada string. Fungsi ini menerima input berupa list of
string dan menghasilkan output berupa representasi binary encoding dari
list tersebut. Jangan lupa membuat fungsi decodernya juga dengan nama
bin_dec_str
4. Buatlah fungsi bernama onehot_enc_str yang berfungsi untuk melakukan
one-hot encoding pada string. Fungsi ini menerima input berupa list of
string dan menghasilkan output berupa representasi one-hot encoding
dari list tersebut. Jangan lupa membuat fungsi decodernya juga dengan
nama onehot_dec_str

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('Iris.csv')

In [ ]:
kelas = df.iloc[:, -1].tolist()
print("Kelas:", kelas)

In [ ]:
def bin_enc_str(list_of_strings):
    encoded_list = []
    for string in list_of_strings:
        binary_string = ' '.join(format(ord(char), '08b') for char in string)
        encoded_list.append(binary_string)
    return encoded_list

In [ ]:
def bin_dec_str(binary_list):
    decoded_list = []
    for binary_string in binary_list:
        decoded_string = ''.join(chr(int(b, 2)) for b in binary_string.split())
        decoded_list.append(decoded_string)
    return decoded_list

In [ ]:
encoded_kelas = bin_enc_str(kelas)
print("Binary Encoded Kelas:", encoded_kelas)

decoded_kelas = bin_dec_str(encoded_kelas)
print("Binary Decoded Kelas:", decoded_kelas)

In [ ]:
def onehot_enc_str(list_of_strings):
    unique_strings = sorted(set(list_of_strings))
    string_to_index = {string: idx for idx, string in enumerate(unique_strings)}

    one_hot_encoded = []
    for string in list_of_strings:
        one_hot = [0] * len(unique_strings)
        one_hot[string_to_index[string]] = 1
        one_hot_encoded.append(one_hot)

    return one_hot_encoded, unique_strings

In [ ]:
def onehot_dec_str(one_hot_encoded, unique_strings):
    decoded_list = []
    for one_hot in one_hot_encoded:
        index = np.argmax(one_hot)
        decoded_list.append(unique_strings[index])

    return decoded_list

In [ ]:
one_hot_encoded, unique_kelas = onehot_enc_str(kelas)
print("One-Hot Encoded Kelas:", one_hot_encoded)
print("Unique Kelas (for decoding):", unique_kelas)

decoded_kelas_onehot = onehot_dec_str(one_hot_encoded, unique_kelas)
print("One-Hot Decoded Kelas:", decoded_kelas_onehot)

In [ ]:
import numpy as np

def bp_fit(X, target, layer_conf, max_epoch, max_error=0.1, learn_rate=0.1, print_per_epoch=100):
    np.random.seed(1)

    nin = [np.empty(i) for i in layer_conf]

    n = [np.empty(j + 1) if i < len(layer_conf) - 1 else np.empty(j)
         for i, j in enumerate(layer_conf)]

    w = [np.random.rand(layer_conf[i] + 1, layer_conf[i + 1])
         for i in range(len(layer_conf) - 1)]

    dw = [np.empty((layer_conf[i] + 1, layer_conf[i + 1]))
          for i in range(len(layer_conf) - 1)]

    d = [np.empty(s) for s in layer_conf[1:]]
    din = [np.empty(s) for s in layer_conf[1:-1]]

    epoch = 0
    mse = 1


    for i in range(0, len(n) - 1):
        n[i][-1] = 1

    while (max_epoch == -1 or epoch < max_epoch) and mse > max_error:
        epoch += 1
        mse = 0

        for r in range(len(X)):
            n[0][:-1] = X[r]

            for L in range(1, len(layer_conf)):
                nin[L] = np.dot(n[L - 1], w[L - 1])
                n[L][:len(nin[L])] = sig(nin[L])

            e = target[r] - n[-1]
            mse += sum(e ** 2)
            d[-1] = e * sigd(nin[-1])
            dw[-1] = learn_rate * d[-1] * n[-2].reshape((-1, 1))

            for L in range(len(layer_conf) - 1, 1, -1):
                din[L - 2] = np.dot(d[L - 1], np.transpose(w[L - 1][:-1]))
                d[L - 2] = din[L - 2] * sigd(nin[L - 1])
                dw[L - 2] = learn_rate * d[L - 2] * n[L - 2].reshape((-1, 1))

            for i in range(len(w)):
                w[i] += dw[i]

        mse /= len(X)

        if print_per_epoch > -1 and epoch % print_per_epoch == 0:
            print(f'Epoch {epoch}, MSE: {mse}')

    return w, epoch, mse

In [ ]:
def bp_predict(X, w):
    n = [np.empty(len(i)) for i in w]
    nin = [np.empty(len(i[0])) for i in w]
    predict = []

    n.append(np.empty(len(w[-1][0])))

    for x in X:
        n[0][:-1] = x

        for L in range(0, len(w)):
            nin[L] = np.dot(n[L], w[L])
            n[L + 1][:len(nin[L])] = sig(nin[L])

        predict.append(n[-1].copy())
    return predict

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

iris = datasets.load_iris()
X = minmax_scale(iris.data)
Y = onehot_enc(iris.target)

X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=.3,random_state=1)
w, ep, mse = bp_fit(X_train, y_train, layer_conf=(4, 3, 3), learn_rate=.1, max_epoch=1000, max_error=.1, print_per_epoch=25)

print(f'Epochs: {ep}, MSE: {mse}')

predict = bp_predict(X_test, w)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test)

print('Output:', predict)
print('True :', y_test)
print('Accuracy:', accuracy)

Epoch 25, MSE: 0.4573000553790559
Epoch 50, MSE: 0.321272689922169
Epoch 75, MSE: 0.2668003450939322
Epoch 100, MSE: 0.19045841193641896
Epoch 125, MSE: 0.13206064032817524
Epoch 150, MSE: 0.10002434429710472
Epochs: 151, MSE: 0.09910797309769231
Output: [0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 2, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 2, 2, 1]
True : [0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 2, 2, 1]
Accuracy: 0.9777777777777777


In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

iris = datasets.load_iris()
X = minmax_scale(iris.data)
Y = onehot_enc(iris.target)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3, random_state=1)
w, ep, mse = bp_fit(X_train, y_train, layer_conf=(4, 2, 3),
learn_rate=.1, max_epoch=100, max_error=.5, print_per_epoch=25)

print (f'Epochs: {ep}, MSE: {mse}')

predict = bp_predict(X_test, w)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test)

print('Output:', predict)
print('True:', y_test)
print('Accuracy:', accuracy)

Epoch 25, MSE: 0.5587148548510855
Epochs: 29, MSE: 0.49569003845261594
Output: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
True: [0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 2, 2, 1]
Accuracy: 0.28888888888888886


In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

iris = datasets.load_iris()
X = minmax_scale(iris.data)
Y = onehot_enc(iris.target)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3, random_state=1)
w, ep, mse = bp_fit(X_train, y_train, layer_conf=(4, 25, 3), learn_rate=.1, max_epoch=10000, max_error=.01, print_per_epoch=25)

print = (f'Epochs: {ep}, MSE: {mse}')

predict = bp_predict(X_test, w)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test)

print('Output:', predict)
print('True :', y_test)
print('Accuracy:', accuracy)

Epoch 25, MSE: 1.9997475602526067
Epoch 50, MSE: 1.9996839107720699
Epoch 75, MSE: 1.9995729256252905
Epoch 100, MSE: 1.9993234166160632
Epoch 125, MSE: 1.998004842894972
Epoch 150, MSE: 1.5034310711672474
Epoch 175, MSE: 0.9882734055300789
Epoch 200, MSE: 0.9358385629366138
Epoch 225, MSE: 0.8962403847767554
Epoch 250, MSE: 0.5720113088188044
Epoch 275, MSE: 0.1718964155883048
Epoch 300, MSE: 0.13551945448567226
Epoch 325, MSE: 0.10976277201695424
Epoch 350, MSE: 0.09159154648169049
Epoch 375, MSE: 0.07929291362007068
Epoch 400, MSE: 0.07095077966556529
Epoch 425, MSE: 0.06511610472051664
Epoch 450, MSE: 0.06088086243803241
Epoch 475, MSE: 0.05769828099112584
Epoch 500, MSE: 0.05523313404391771
Epoch 525, MSE: 0.05327321999520479
Epoch 550, MSE: 0.051680019060663
Epoch 575, MSE: 0.05036062409674887
Epoch 600, MSE: 0.0492511427801703
Epoch 625, MSE: 0.04830653743243751
Epoch 650, MSE: 0.0474942479018858
Epoch 675, MSE: 0.0467901200965703
Epoch 700, MSE: 0.046175774075554316
Epoch 725, 

TypeError: 'str' object is not callable

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

del print

iris = datasets.load_iris()
X = minmax_scale(iris.data)
Y = onehot_enc(iris.target)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3, random_state=1)
w, ep, mse = bp_fit(X_train, y_train, layer_conf=(4, 10, 3), learn_rate=.1, max_epoch=2000, max_error=.01, print_per_epoch=25)

print = (f'Epochs: {ep}, MSE: {mse}')

predict = bp_predict(X_test, w)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test)

print('Output:', predict)

Epoch 25, MSE: 0.383776096514961
Epoch 50, MSE: 0.2938290580342846
Epoch 75, MSE: 0.23051680773921684
Epoch 100, MSE: 0.1641746565691314
Epoch 125, MSE: 0.12054245285529827
Epoch 150, MSE: 0.09528092568898881
Epoch 175, MSE: 0.08037329840572681
Epoch 200, MSE: 0.07110175933017954
Epoch 225, MSE: 0.06496455009618983
Epoch 250, MSE: 0.060650291850340744
Epoch 275, MSE: 0.057455956978168486
Epoch 300, MSE: 0.05498915006560867
Epoch 325, MSE: 0.05302027463743097
Epoch 350, MSE: 0.051408353869327045
Epoch 375, MSE: 0.05006270656010097
Epoch 400, MSE: 0.04892241143572634
Epoch 425, MSE: 0.04794487414778065
Epoch 450, MSE: 0.04709922028747376
Epoch 475, MSE: 0.04636234110506702
Epoch 500, MSE: 0.045716448784807844
Epoch 525, MSE: 0.04514751871364106
Epoch 550, MSE: 0.04464426822008323
Epoch 575, MSE: 0.04419746831397594
Epoch 600, MSE: 0.04379946709053554
Epoch 625, MSE: 0.043443850731851
Epoch 650, MSE: 0.04312519596744861
Epoch 675, MSE: 0.042838884724883404
Epoch 700, MSE: 0.04258096207820

TypeError: 'str' object is not callable

In [ ]:
del print
print('True :', y_test)

True : [0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 2, 2, 1]


In [ ]:
print('Accuracy:', accuracy)

Accuracy: 1.0
